In [2]:
import pandas as pd

# Ensemble de données sur le vin

Cet ensemble de données est le résultat d'une analyse chimique de vins cultivés dans la même région d'Italie, mais issus de trois cépages différents. Il comprend 178 échantillons et 13 attributs d'analyse chimique. Les colonnes sont numérotées de 1 à 13 et correspondent aux éléments suivants :

Alcool : La teneur en alcool du vin.
Acide malique : un type d’acide présent dans le vin.
Cendres : Mesure de la teneur en « cendres » du vin, qui correspond à la matière inorganique restant après évaporation et incinération.
Alcalinité des cendres : mesure de l'alcalinité du vin, en particulier de l'alcalinité des cendres.
Magnésium : La teneur en magnésium du vin.
Phénols totaux : Mesure de la teneur totale en phénols du vin.
Flavonoïdes : Un sous-ensemble de phénols, les flavonoïdes sont un type d'antioxydant.
Phénols non flavonoïdes : Cet attribut mesure les phénols non flavonoïdes présents dans le vin.
Proanthocyanidines : Un autre groupe de phénols.
Intensité de la couleur : mesure de l'intensité de la couleur du vin.
Teinte : Attribut du vin lié à sa couleur.
OD280/OD315 des vins dilués : Il s'agit d'une mesure de l'absorbance du vin aux longueurs d'onde de 280 nm et 315 nm après dilution.
Proline : Mesure de la teneur en proline (un acide aminé) du vin.

In [3]:
# url   data_vin
wine_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"

In [4]:
wine_data = pd.read_csv(wine_url, header= None ) 

In [41]:
import pandas as pd
from urllib.parse import urlparse
from pathlib import PurePosixPath
from typing import Any, Optional, Sequence

def load_data(
    url: str,
    *,
    sep: Optional[str] = None,        # None => détection auto (engine='python')
    header: Optional[int | str] = "infer",
    names: Optional[Sequence[str]] = None,
    **read_kwargs: Any
) -> pd.DataFrame:
    """
    Charge un dataset depuis une URL/chemin en détectant le format.
    Gère CSV/TSV/TXT/DATA/DAT/JSON/JSONL/Excel/Parquet/Feather/Pickle.
    Particularité : .data/.dat (UCI) sont traités comme CSV.
    """

    if not isinstance(url, str) or not url.strip():
        raise TypeError("`url` doit être une chaîne non vide.")

    path = urlparse(url).path
    suffixes = [s.lower() for s in PurePosixPath(path).suffixes]  # ex: ['.csv', '.gz']
    joined = "".join(suffixes)

    def has(*patterns: str) -> bool:
        return any(p in joined for p in patterns)

    # --- petit helper pour CSV avec gestion propre de l'engine ---
    def _read_csv(u: str, sep_arg: Optional[str]):
        csv_kwargs = dict(header=header, names=names, **read_kwargs)
        if sep_arg is None:
            # Sniff du séparateur => engine='python', surtout PAS low_memory
            return pd.read_csv(u, sep=None, engine="python", **csv_kwargs)
        else:
            # Engine par défaut (C) => on peut utiliser low_memory
            return pd.read_csv(u, sep=sep_arg, low_memory=False, **csv_kwargs)

    try:
        # ===== Formats par extension =====
        if has(".csv", ".txt", ".data", ".dat"):
            return _read_csv(url, sep)

        if has(".tsv"):
            return _read_csv(url, "\t")

        if has(".xlsx", ".xls"):
            return pd.read_excel(url, header=0 if header == "infer" else header, **read_kwargs)

        if has(".jsonl"):
            return pd.read_json(url, lines=True, **read_kwargs)
        if has(".json"):
            return pd.read_json(url, **read_kwargs)

        if has(".parquet"):
            return pd.read_parquet(url, **read_kwargs)
        if has(".feather"):
            return pd.read_feather(url, **read_kwargs)

        if has(".pkl", ".pickle"):
            return pd.read_pickle(url, **read_kwargs)

        # ===== Fallbacks si extension inconnue =====
        # 1) CSV sniff (engine='python')
        try:
            return _read_csv(url, None)
        except Exception:
            pass

        # 2) TSV/whitespace
        try:
            return pd.read_csv(url, sep=r"\s+", engine="python",
                               header=header, names=names, **read_kwargs)
        except Exception:
            pass

        # 3) JSONL → JSON
        for kw in (dict(lines=True), dict()):
            try:
                return pd.read_json(url, **kw, **read_kwargs)
            except Exception:
                continue

        # 4) Excel
        try:
            return pd.read_excel(url, header=0 if header == "infer" else header, **read_kwargs)
        except Exception:
            pass

        raise ValueError(
            "Format non reconnu et échec des fallbacks "
            "(CSV/whitespace/JSON/Excel). Spécifiez `sep`, `header`, `names` "
            "ou vérifiez la ressource."
        )

    except ImportError as e:
        raise ImportError(
            f"Moteur manquant pour ce format : {e}. "
            "Installez le paquet requis (ex. `pip install pyarrow fastparquet openpyxl`)."
        ) from e
    except Exception as e:
        raise ValueError(f"Échec de lecture de '{url}': {e}") from e


In [42]:
load_data(wine_url)

,1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
173,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
174,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
175,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [13]:
from package_exploration_data.valeur_manquante import ValeurManquante

ValeurManquante.valeur_manquante(wine_data)

'La base de donnée ne contient pas de valeur manquante'

In [15]:
wine_data.duplicated().sum()

np.int64(0)

In [16]:
from package_exploration_data.doublon import Doublon

In [17]:
Doublon().nombre_lignes_dupliquees(wine_data)

0

In [18]:
from imputation.imputation import MoyenMedianMode

In [25]:
MoyenMedianMode().calcul_imputation(wine_data, 0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840
